In [43]:
from IPython.display import HTML
HTML('<img src="pics/sf.gif" width="800">')

In [44]:
# import python packages
import pandas as pd
import os
# https://github.com/SohierDane/BigQuery_Helper
from bq_helper import BigQueryHelper


In [45]:
# set enviroment variable for Google Cloud API
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/Users/tfox2791/Downloads/REMOVED KEY FOR PRIVACY.json"

In [46]:
# use Big Query Helper to access database
bq_assistant = BigQueryHelper("bigquery-public-data", "san_francisco")

QUERY = "SELECT * FROM `bigquery-public-data.san_francisco.sfpd_incidents`"

In [47]:
# list the tables in db
bq_assistant.list_tables()

['311_service_requests',
 'bikeshare_stations',
 'bikeshare_status',
 'bikeshare_trips',
 'film_locations',
 'sffd_service_calls',
 'sfpd_incidents',
 'street_trees']

In [48]:
# convert query into pandas dataframe 
df = bq_assistant.query_to_pandas(QUERY)

In [49]:
# take a peak at data
df.head()

,unique_key,category,descript,dayofweek,pddistrict,resolution,address,longitude,latitude,location,pdid,timestamp
0,166018573,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Sunday,,NONE,100 Block of VELASCO AV,-122.413352,37.708202,"(37.70820245849022, -122.4133519852842)",16601857306244,2016-01-17 23:54:00+00:00
1,160874408,DRIVING UNDER THE INFLUENCE,DRIVING WHILE UNDER THE INFLUENCE OF DRUGS,Thursday,PARK,NONE,DUBOCE AV / NOE ST,-122.433575,37.769177,"(37.769176747627725, -122.43357509728241)",16087440865060,2016-10-27 04:15:00+00:00
2,120574836,ROBBERY,ATTEMPTED ROBBERY ON THE STREET WITH A KNIFE,Saturday,PARK,NONE,HAIGHT ST / DIVISADERO ST,-122.437049,37.771268,"(37.7712678186367, -122.437048523435)",12057483603412,2012-07-21 05:55:00+00:00
3,160573898,SECONDARY CODES,JUVENILE INVOLVED,Saturday,PARK,NONE,1200 Block of PAGE ST,-122.441394,37.771727,"(37.77172731417711, -122.44139370111428)",16057389815500,2016-07-16 11:15:00+00:00
4,40502947,DRUG/NARCOTIC,POSSESSION OF COCAINE,Sunday,PARK,"ARREST, BOOKED",1300 Block of FULTON ST,-122.439038,37.776802,"(37.7768021540039, -122.439037573428)",4050294716620,2004-05-02 11:59:00+00:00


In [50]:
# data info
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2170785 entries, 0 to 2170784
Data columns (total 12 columns):
unique_key    int64
category      object
descript      object
dayofweek     object
pddistrict    object
resolution    object
address       object
longitude     float64
latitude      float64
location      object
pdid          int64
timestamp     datetime64[ns, UTC]
dtypes: datetime64[ns, UTC](1), float64(2), int64(2), object(7)
memory usage: 198.7+ MB


In [51]:
# describe data, rounded to remove scientific notation
round(df.describe(include='all'),0)

,unique_key,category,descript,dayofweek,pddistrict,resolution,address,longitude,latitude,location,pdid,timestamp
count,2170785.0,2170785,2170785,2170785,2170785,2170785,2170785,2170785.0,2170785.0,2170785,2.170785e+06,2170785
unique,NaN,39,915,7,11,17,25123,NaN,NaN,60768,NaN,960696
top,NaN,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Friday,SOUTHERN,NONE,800 Block of BRYANT ST,NaN,NaN,"(37.775420706711, -122.403404791479)",NaN,2011-01-01 00:01:00+00:00
freq,NaN,467657,173079,331007,391669,1356705,64271,NaN,NaN,55669,NaN,185
mean,102498901.0,NaN,NaN,NaN,NaN,NaN,NaN,-122.0,38.0,NaN,1.024989e+13,NaN
std,45243360.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,4.524336e+12,NaN
min,3979.0,NaN,NaN,NaN,NaN,NaN,NaN,-123.0,38.0,NaN,3.979630e+08,NaN
25%,61120924.0,NaN,NaN,NaN,NaN,NaN,NaN,-122.0,38.0,NaN,6.112092e+12,NaN
50%,100938092.0,NaN,NaN,NaN,NaN,NaN,NaN,-122.0,38.0,NaN,1.009381e+13,NaN
75%,140638923.0,NaN,NaN,NaN,NaN,NaN,NaN,-122.0,38.0,NaN,1.406389e+13,NaN


In [52]:
# export and save file to csv
df.to_csv('sfpd.csv', sep='\t')

In [53]:
# add hour of day timestamp to df
df['hour_of_day'] = df.timestamp.dt.hour

In [54]:
# group data by category, unstack, reset the index and fillna's

sfpd_by_hour_df = df.groupby(['category']).hour_of_day.value_counts()
sfpd_by_hour_df = sfpd_by_hour_df.unstack(level=0)
sfpd_by_hour_df = pd.DataFrame(sfpd_by_hour_df).reset_index()
sfpd_by_hour_df.fillna(0, inplace=True)
sfpd_by_hour_df.head()

category,hour_of_day,ARSON,ASSAULT,BAD CHECKS,BRIBERY,BURGLARY,DISORDERLY CONDUCT,DRIVING UNDER THE INFLUENCE,DRUG/NARCOTIC,DRUNKENNESS,...,"SEX OFFENSES, NON FORCIBLE",STOLEN PROPERTY,SUICIDE,SUSPICIOUS OCC,TREA,TRESPASS,VANDALISM,VEHICLE THEFT,WARRANTS,WEAPON LAWS
0,0,249.0,10644.0,159.0,43.0,3399.0,523.0,714.0,3744.0,821.0,...,80.0,546.0,46.0,5111.0,0.0,557.0,6412.0,4716.0,4007.0,1125.0
1,1,238.0,9089.0,3.0,26.0,2200.0,383.0,650.0,2274.0,780.0,...,6.0,388.0,43.0,2173.0,0.0,399.0,4484.0,3215.0,2731.0,830.0
2,2,257.0,8264.0,2.0,27.0,2374.0,343.0,645.0,1842.0,658.0,...,2.0,325.0,30.0,1980.0,2.0,438.0,4239.0,2550.0,2274.0,731.0
3,3,257.0,4207.0,3.0,20.0,2419.0,216.0,288.0,1248.0,259.0,...,2.0,248.0,23.0,1376.0,0.0,360.0,2835.0,1581.0,1755.0,444.0
4,4,216.0,2652.0,3.0,9.0,2154.0,159.0,108.0,909.0,133.0,...,2.0,225.0,15.0,937.0,0.0,261.0,1900.0,1220.0,1347.0,312.0


In [55]:
# describe new df
round(sfpd_by_hour_df.describe(),0)

category,hour_of_day,ARSON,ASSAULT,BAD CHECKS,BRIBERY,BURGLARY,DISORDERLY CONDUCT,DRIVING UNDER THE INFLUENCE,DRUG/NARCOTIC,DRUNKENNESS,...,"SEX OFFENSES, NON FORCIBLE",STOLEN PROPERTY,SUICIDE,SUSPICIOUS OCC,TREA,TRESPASS,VANDALISM,VEHICLE THEFT,WARRANTS,WEAPON LAWS
count,24.0,24.0,24.0,24.0,24.0,24.0,24.0,24.0,24.0,24.0,...,24.0,24.0,24.0,24.0,24.0,24.0,24.0,24.0,24.0,24.0
mean,12.0,160.0,7933.0,38.0,33.0,3730.0,415.0,233.0,4928.0,406.0,...,18.0,485.0,53.0,3284.0,1.0,790.0,4726.0,5217.0,4158.0,904.0
std,7.0,62.0,2716.0,47.0,14.0,1205.0,198.0,229.0,2670.0,241.0,...,16.0,184.0,19.0,1357.0,1.0,276.0,2084.0,2876.0,1650.0,366.0
min,0.0,78.0,2116.0,2.0,7.0,1963.0,159.0,26.0,475.0,45.0,...,2.0,172.0,15.0,762.0,0.0,261.0,1573.0,1220.0,952.0,169.0
25%,6.0,113.0,6869.0,4.0,24.0,2935.0,306.0,58.0,2830.0,194.0,...,6.0,350.0,41.0,2129.0,0.0,552.0,3368.0,3190.0,3077.0,697.0
50%,12.0,150.0,9067.0,20.0,37.0,3726.0,380.0,126.0,4991.0,344.0,...,16.0,508.0,58.0,3575.0,1.0,858.0,4362.0,4481.0,4384.0,1062.0
75%,17.0,220.0,10003.0,50.0,44.0,4324.0,418.0,310.0,6985.0,582.0,...,19.0,631.0,70.0,4214.0,1.0,980.0,6931.0,8321.0,5478.0,1198.0
max,23.0,257.0,10644.0,186.0,55.0,6279.0,1000.0,714.0,8907.0,821.0,...,80.0,761.0,78.0,5880.0,2.0,1249.0,8227.0,10220.0,6495.0,1338.0


In [56]:
# save new df to csv for plotting with Bokeh
sfpd_by_hour_df.to_csv('sfpd_by_hour_df.csv', sep='\t')